In [ ]:
import json

def validate_json(json_str):
    try:
        json.loads(json_str)
        print("JSON is valid.")
    except json.JSONDecodeError as e:
        print("JSON is not valid. Error:", e)

# Read JSON data from the file
file_path = "/content/drive/MyDrive/Colab Notebooks/IITP_RESEARCH_REDDIT/Dataset_reddit/try.json"  # Update with the path to your JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = file.read()

# Validate the JSON data
validate_json(json_data)

JSON is valid.


In [ ]:
!pip install fastembed
!pip install qdrant-client[fastembed]
# !pip install qdrant_client.client
# !apt-get install -y cmake build-essential libssl-dev
# !pip install git+https://github.com/qdrant/qdrant_client.git
%pip install git+https://github.com/qdrant/qdrant_client.git


  Cloning https://github.com/qdrant/qdrant_client.git to /tmp/pip-req-build-n55xt9nc
  Running command git clone --filter=blob:none --quiet https://github.com/qdrant/qdrant_client.git /tmp/pip-req-build-n55xt9nc
  Resolved https://github.com/qdrant/qdrant_client.git to commit eda201a1dbf1bbc67415f8437a5619f6f83e8ac6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from fastembed.embedding import FlagEmbedding as Embedding
from typing import List
import numpy as np
import qdrant_client

In [ ]:
import json
from typing import List
import numpy as np
from fastembed.embedding import FlagEmbedding as Embedding
from qdrant_client import QdrantClient

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def extract_text_data(data):
    documents = []
    for thread in data["threads"]:
        documents.append(thread["caption"])  # Adding post captions
        for comment in thread["comments"]:
            if "body" in comment:
                documents.append(comment["body"])  # Adding comment bodies if "body" exists
    return documents

def embed_documents(documents):
    embedding_model = Embedding(model_name="BAAI/bge-base-en", max_length=512)
    embeddings = embedding_model.embed(documents)
    return embeddings

# Load JSON data
file_path = "/content/drive/MyDrive/Colab Notebooks/IITP_RESEARCH_REDDIT/Dataset_reddit/try.json"  # Update with your file path
json_data = load_json(file_path)

# Extract text data from JSON
documents = extract_text_data(json_data)

# Generate embeddings for text data
embeddings = embed_documents(documents)

# Initialize Qdrant client
client = QdrantClient(":memory:")

# Prepare metadata and IDs
metadata = [{"source": "Reddit", "embedding": embedding.tolist()} for embedding in embeddings]
ids = list(range(len(documents)))

# Add documents along with their embeddings to Qdrant
client.add(
    collection_name="reddit_collection",
    documents=documents,
    metadata=metadata,
    ids=ids
)

# Perform a search query
search_result = client.query(
    collection_name="reddit_collection",
    query_text="Your query text here"
)
print(search_result)


100%|██████████| 77.7M/77.7M [00:04<00:00, 18.0MiB/s]


[QueryResponse(id=0, embedding=None, metadata={'document': 'would you put the sectional on the other side??', 'source': 'Reddit', 'embedding': [0.025046436116099358, -0.030460473150014877, 0.024093249812722206, -0.0006550318212248385, 0.04123987257480621, -0.005473040044307709, 0.05103541910648346, 0.011729778721928596, -0.012303553521633148, -0.07051204144954681, 0.02232714369893074, -0.009095264598727226, -0.07625226676464081, 0.028925837948918343, -0.03974180296063423, 0.07128167152404785, 0.04493110626935959, 0.007948679849505424, -0.002698581200093031, -0.005767319351434708, 0.017364829778671265, -0.017976850271224976, -0.044853854924440384, 0.0061987112276256084, 0.03358153626322746, -0.03840861842036247, 0.027429480105638504, 0.013944603502750397, -0.05457326024770737, -0.03280564770102501, 0.015780076384544373, -0.03158191591501236, -0.03384438902139664, -0.06226303428411484, 0.017637476325035095, -0.043451473116874695, -0.020857317373156548, -0.009214025922119617, -0.034567877

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Reference summary
reference_summary = "The OP wanted to know which wall they should place their sectional couch up against in their living room. \nMost commenters agreed that the seating should stay by the windows. \nOne commenter said to get a smaller couch, but OP said they can't."

# Generated summary
generated_summary = "Absolutely do not put the tv in front of your beautiful windows. This sofa won't work in this room no matter what you do so you might as well leave it where it is. Possibly hang the tv on the wall instead of crowding the room even more with more (dark) furniture."

# Initialize the Rouge object
rouge = Rouge()

# Compute ROUGE scores
rouge_scores = rouge.get_scores(generated_summary, reference_summary, avg=True)

# Print ROUGE scores
print(f"ROUGE-1 Score: {rouge_scores['rouge-1']['f'] * 100}")
print(f"ROUGE-2 Score: {rouge_scores['rouge-2']['f'] * 100}")
print(f"ROUGE-L Score: {rouge_scores['rouge-l']['f'] * 100}")

ROUGE-1 Score: 12.98701248912129
ROUGE-2 Score: 0.0
ROUGE-L Score: 12.98701248912129


In [ ]:
# from fastembed.embedding import FlagEmbedding as Embedding

# documents: List[str] = [
#     "passage: Hello, World!",
#     "query: Hello, World!", # these are two different embedding
#     "passage: This is an example passage.",
#     "fastembed is supported by and maintained by Qdrant." # You can leave out the prefix but it's recommended
# ]
# embedding_model = Embedding(model_name="BAAI/bge-base-en", max_length=512)
# embeddings: List[np.ndarray] = embedding_model.embed(documents) # If you use

In [ ]:
# !pip install qdrant-client[fastembed]

In [ ]:
# from qdrant_client import QdrantClient

# # Initialize the client
# client = QdrantClient(":memory:")  # or QdrantClient(path="path/to/db")

# # Prepare your documents, metadata, and IDs
# docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]
# metadata = [
#     {"source": "Langchain-docs"},
#     {"source": "Linkedin-docs"},
# ]
# ids = [42, 2]

# # Use the new add method
# client.add(
#     collection_name="demo_collection",
#     documents=docs,
#     metadata=metadata,
#     ids=ids
# )

# search_result = client.query(
#     collection_name="demo_collection",
#     query_text="This is a query document"
# )
# print(search_result)